In [ ]:
import sys
print(f"Python version: {sys.version}")

!pip install torch-geometric -q
!pip install deepchem -q

print("\nInstalling RDKit...")
!pip install rdkit

print("\n✅ All packages installed!")

Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]

Installing RDKit...

✅ All packages installed!


In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data, Batch
import deepchem as dc
import math
import time

print("✅ All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"DeepChem version: {dc.__version__}")
print(f"GPU available: {torch.cuda.is_available()}")

✅ All imports successful!
PyTorch version: 2.9.0+cu126
DeepChem version: 2.5.0
GPU available: True


In [ ]:
class RadialBasis(nn.Module):
    """Bessel radial basis for distance encoding"""

    def __init__(self, num_basis=8, cutoff=5.0):
        super().__init__()
        self.cutoff = cutoff
        self.frequencies = nn.Parameter(
            torch.arange(1, num_basis + 1) * math.pi / cutoff,
            requires_grad=False
        )

    def forward(self, distances):
        """Encode distances as basis functions"""
        # Cutoff envelope
        envelope = torch.where(
            distances < self.cutoff,
            torch.cos(distances * math.pi / (2 * self.cutoff)) ** 2,
            torch.zeros_like(distances)
        )

        # Bessel basis
        d = distances.unsqueeze(-1)
        basis = torch.sin(self.frequencies * d) / d

        return basis * envelope.unsqueeze(-1)

print("✅ RadialBasis defined")

✅ RadialBasis defined


In [ ]:
class MACEInteraction(MessagePassing):
    """MACE message passing interaction layer"""

    def __init__(self, hidden_dim, num_basis):
        super().__init__(aggr='add')

        # Message network
        self.message_net = nn.Sequential(
            nn.Linear(hidden_dim * 2 + num_basis, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

        # Update network
        self.update_net = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

    def forward(self, x, edge_index, edge_attr):
        """Message passing step"""
        out = self.propagate(edge_index, x=x, edge_attr=edge_attr)
        out = self.update_net(torch.cat([x, out], dim=-1))
        return out

    def message(self, x_i, x_j, edge_attr):
        """Construct messages"""
        msg_input = torch.cat([x_i, x_j, edge_attr], dim=-1)
        return self.message_net(msg_input)

print("✅ MACEInteraction defined")

✅ MACEInteraction defined


In [ ]:
class MACE(nn.Module):
    """MACE: Multi-Atomic Cluster Expansion Neural Network"""

    def __init__(
        self,
        num_elements=100,
        hidden_dim=64,
        num_interactions=2,
        num_basis=8,
        cutoff=5.0
    ):
        super().__init__()

        self.cutoff = cutoff

        # Atom embedding
        self.atom_embedding = nn.Embedding(num_elements, hidden_dim)

        # Radial basis
        self.radial_basis = RadialBasis(num_basis, cutoff)

        # Interaction layers
        self.interactions = nn.ModuleList([
            MACEInteraction(hidden_dim, num_basis)
            for _ in range(num_interactions)
        ])

        # Energy head
        self.energy_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, z, pos, edge_index, batch=None):
        """
        Forward pass

        Args:
            z: (N,) atomic numbers
            pos: (N, 3) atomic positions
            edge_index: (2, E) edge indices
            batch: (N,) batch assignment

        Returns:
            energy: (batch_size,) or scalar
            forces: (N, 3) atomic forces
        """
        # Edge features
        row, col = edge_index
        edge_vec = pos[row] - pos[col]
        edge_dist = edge_vec.norm(dim=-1)
        edge_attr = self.radial_basis(edge_dist)

        # Embed atoms
        x = self.atom_embedding(z)

        # Message passing with residual connections
        for interaction in self.interactions:
            x = x + interaction(x, edge_index, edge_attr)

        # Predict atomic energies
        atomic_energies = self.energy_head(x)

        # Sum to molecular energy
        if batch is None:
            energy = atomic_energies.sum()
        else:
            from torch_geometric.utils import scatter
            energy = scatter(atomic_energies, batch, dim=0, reduce='sum')

        energy = energy.squeeze(-1)

        # Compute forces
        forces = None
        if pos.requires_grad:
            forces = -torch.autograd.grad(
                energy.sum(), pos, create_graph=True
            )[0]

        return energy, forces

print("✅ MACE neural network defined")

✅ MACE neural network defined


In [ ]:
print("="*70)
print("TESTING MACE ON H2O MOLECULE")
print("="*70)

# Create water molecule
z = torch.tensor([8, 1, 1], dtype=torch.long)  # O, H, H
pos = torch.tensor([
    [0.0, 0.0, 0.0],
    [0.96, 0.0, 0.0],
    [-0.24, 0.93, 0.0]
], requires_grad=True)

edge_index = torch.tensor([
    [0, 1, 0, 2, 1, 2],
    [1, 0, 2, 0, 2, 1]
], dtype=torch.long)

# Create MACE
mace = MACE(hidden_dim=32, num_interactions=2)

# Forward pass
energy, forces = mace(z, pos, edge_index)

print(f"\n✅ MACE works!")
print(f"   Input: H2O (3 atoms)")
print(f"   Energy: {energy.item():.6f}")
print(f"   Forces shape: {forces.shape}")
print(f"   Force magnitudes: {forces.norm(dim=1)}")
print("\n🎉 MACE neural network tested successfully!")

TESTING MACE ON H2O MOLECULE

✅ MACE works!
   Input: H2O (3 atoms)
   Energy: 0.641665
   Forces shape: torch.Size([3, 3])
   Force magnitudes: tensor([0.0164, 0.0132, 0.0132], grad_fn=<LinalgVectorNormBackward0>)

🎉 MACE neural network tested successfully!


In [ ]:
from deepchem.models.losses import Loss

class MACELoss(Loss):
    """Loss function for MACE"""

    def __init__(self, energy_weight=1.0):
        super().__init__()
        self.energy_weight = energy_weight

    def _create_pytorch_loss(self):
        """Required by DeepChem - return a PyTorch loss function"""
        # Return nn.MSELoss directly
        return torch.nn.MSELoss()

print("✅ MACELoss defined (using PyTorch MSELoss)")

✅ MACELoss defined (using PyTorch MSELoss)


In [ ]:
from deepchem.models import TorchModel
import numpy as np

class MACEWrapper(nn.Module):
    """Wrapper that unpacks PyG batches for MACE"""

    def __init__(self, mace_net):
        super().__init__()
        self.mace_net = mace_net

    def forward(self, inputs):
        """Unpack PyG batch and call MACE"""
        pyg_batch = inputs[0]
        energy, forces = self.mace_net(
            z=pyg_batch.z,
            pos=pyg_batch.pos,
            edge_index=pyg_batch.edge_index,
            batch=pyg_batch.batch
        )
        if energy.dim() == 0:
            energy = energy.unsqueeze(0)
        return energy


class MACEModel(TorchModel):
    """MACE integrated with DeepChem - COMPLETE FIX"""

    def __init__(
        self,
        num_elements=100,
        hidden_dim=64,
        num_interactions=2,
        learning_rate=0.001,
        **kwargs
    ):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        kwargs['device'] = device

        print(f"🔥 Using device: {device}")

        mace_net = MACE(
            num_elements=num_elements,
            hidden_dim=hidden_dim,
            num_interactions=num_interactions
        )

        wrapper = MACEWrapper(mace_net)

        # SET batch_size in kwargs
        if 'batch_size' not in kwargs:
            kwargs['batch_size'] = 32

        self._internal_batch_size = kwargs['batch_size']

        super().__init__(
            model=wrapper,
            loss=MACELoss(),
            output_types=['prediction'],
            n_tasks=1,
            learning_rate=learning_rate,
            **kwargs
        )

    def _prepare_batch(self, batch):
        """Convert DeepChem batch to PyG format"""
        if len(batch) == 4:
            inputs, labels, weights, ids = batch
        elif len(batch) == 3:
            inputs, labels, weights = batch
        elif len(batch) == 2:
            inputs, labels = batch
            weights = None
        else:
            raise ValueError(f"Unexpected batch length: {len(batch)}")

        if labels is None:
            y = None
        else:
            y = labels[0] if isinstance(labels, list) else labels
            if hasattr(y, 'shape') and len(y.shape) > 1:
                y = y[:, 0]

        X = inputs[0] if isinstance(inputs, list) else inputs
        device = next(self.model.parameters()).device

        data_list = []
        for i in range(len(X)):
            graph = X[i]
            num_atoms = graph.get_num_atoms()
            atom_features = graph.get_atom_features()

            z = torch.tensor(atom_features[:, 0], dtype=torch.long, device=device)
            pos = torch.randn(num_atoms, 3, device=device)
            pos.requires_grad = True

            adj_list = graph.get_adjacency_list()
            edge_index = []
            for atom_idx, neighbors in enumerate(adj_list):
                for neighbor_idx in neighbors:
                    edge_index.append([atom_idx, neighbor_idx])

            if not edge_index:
                for j in range(num_atoms):
                    for k in range(num_atoms):
                        if j != k:
                            edge_index.append([j, k])

            edge_index = torch.tensor(edge_index, dtype=torch.long, device=device).t().contiguous()

            if y is not None:
                y_val = torch.tensor([y[i]], dtype=torch.float32, device=device)
            else:
                y_val = torch.zeros(1, dtype=torch.float32, device=device)

            data = Data(z=z, pos=pos, edge_index=edge_index, y=y_val)
            data_list.append(data)

        pyg_batch = Batch.from_data_list(data_list)

        if weights is None:
            weights = [torch.ones(len(X), dtype=torch.float32, device=device)]
        elif isinstance(weights, np.ndarray):
            weights = [torch.tensor(weights, dtype=torch.float32, device=device)]
        elif not isinstance(weights[0], torch.Tensor):
            weights = [torch.tensor(weights[0], dtype=torch.float32, device=device)]
        else:
            weights = [weights[0].to(device)]

        return ([pyg_batch], [pyg_batch.y], weights)

    def predict(self, dataset, transformers=[], output_types=None):
        """Override predict to handle batching correctly"""
        all_predictions = []

        # Process in batches
        for batch in dataset.iterbatches(batch_size=self._internal_batch_size, deterministic=True):
            X_batch = batch[0]

            self.model.eval()
            with torch.no_grad():
                inputs, _, _ = self._prepare_batch((X_batch, None, None))
                outputs = self.model(inputs)

                # Move to CPU
                if isinstance(outputs, torch.Tensor):
                    outputs = outputs.cpu().detach().numpy()

                all_predictions.append(outputs)

        # Concatenate all batches
        result = np.concatenate(all_predictions)
        return result

print("✅ MACEModel with predict override!")

✅ MACEModel with predict override!


In [ ]:
# Test prediction
import deepchem as dc

tasks, datasets, _ = dc.molnet.load_qm9(featurizer='GraphConv', splitter='random', reload=True)
valid_full = datasets[1]

valid_y_flat = valid_full.y[:5, 0]
valid_w = valid_full.w[:5, 0:1]

valid_energy = dc.data.NumpyDataset(
    X=valid_full.X[:5],
    y=valid_y_flat,
    w=valid_w,
    ids=valid_full.ids[:5]
)

mace_model = MACEModel(hidden_dim=32, num_interactions=2, batch_size=10)

y_pred = mace_model.predict(valid_energy)

print(f"Dataset y shape: {valid_energy.y.shape}")
print(f"Prediction shape: {y_pred.shape}")
print(f"✅ Shapes match: {valid_energy.y.shape == y_pred.shape}")

🔥 Using device: cuda
Dataset y shape: (5,)
Prediction shape: (5,)
✅ Shapes match: True


In [ ]:
print("="*70)
print("🔥 DAY 1 FINAL - Y KO BHI FLATTEN KARO")
print("="*70)

import deepchem as dc
import numpy as np
import time

# Load
print("\n1. Loading QM9...")
tasks, datasets, _ = dc.molnet.load_qm9(featurizer='GraphConv', splitter='random', reload=True)
train_full, valid_full, _ = datasets


train_y_flat = train_full.y[:100, 0]
valid_y_flat = valid_full.y[:20, 0]

train_w = train_full.w[:100, 0:1] if len(train_full.w.shape) == 2 else train_full.w[:100].reshape(-1, 1)
valid_w = valid_full.w[:20, 0:1] if len(valid_full.w.shape) == 2 else valid_full.w[:20].reshape(-1, 1)

print(f"   Train y shape: {train_y_flat.shape}")
print(f"   Valid y shape: {valid_y_flat.shape}")

train_energy = dc.data.NumpyDataset(
    X=train_full.X[:100],
    y=train_y_flat,
    w=train_w,
    ids=train_full.ids[:100]
)
valid_energy = dc.data.NumpyDataset(
    X=valid_full.X[:20],
    y=valid_y_flat,
    w=valid_w,
    ids=valid_full.ids[:20]
)

# Model
print("\n2. Creating MACE...")
mace_model = MACEModel(hidden_dim=32, num_interactions=2, batch_size=10)

# Train
print("\n3. Training...")
start = time.time()
mace_model.fit(train_energy, nb_epoch=10)
print(f"   ✅ {time.time()-start:.1f}s")

# Evaluate
print("\n4. Evaluating...")
metric = dc.metrics.Metric(dc.metrics.mean_absolute_error)
score = mace_model.evaluate(valid_energy, [metric])

print(f"\n📊 MAE: {score['mean_absolute_error']:.3f} kcal/mol")
print("\n🎉🎉🎉 DAY 1 COMPLETE! 🎉🎉🎉")
print("="*70)

🔥 DAY 1 FINAL - Y KO BHI FLATTEN KARO

1. Loading QM9...
   Train y shape: (100,)
   Valid y shape: (20,)

2. Creating MACE...
🔥 Using device: cuda

3. Training...


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


   ✅ 1.2s

4. Evaluating...

📊 MAE: 0.705 kcal/mol

🎉🎉🎉 DAY 1 COMPLETE! 🎉🎉🎉
